# Übung 1

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data'
cols = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','TGT']
boston = pd.read_csv(url,sep=' ',skipinitialspace=True,header=None,names=cols,index_col=False)
boston.head()

## Aufgabe 1

### a)
Führen Sie für diesen Datensatz eine explorative Analyse wie in der Vorlesung gezeigt
mithilfe eines IPython-Notebooks und den Paketen Pandas und Numpy durch.

In [ ]:
print(boston.info())

In [ ]:
boston.describe()

#### Aufbereitung der Daten

#### Korrelation

### b)
Beantworten Sie anhand der Darstellung der Streumatrix folgende Fragen:
Welche der Variablen sind
kategorisch?


1. CHAS ist kategorisch (nur 0 oder 1)
2. RAD ist kategorisch (Index von 1 bis 24)

In [ ]:
pd.plotting.scatter_matrix(boston, figsize=(20,20), marker='o');

Welche der Variablen eignen sich gut zur Vorhersage des Hauspreises und
warum?

In [ ]:
boston.corr()

1. LSTAT (neg. korreliert)
2. RM (pos. korreliert)
3. PTRATIO ( neg. korreliert)
4. INDUS (neg. korreliert)
5. TAX (neg. korreliert)
6. NOX (neg. korreliert)

Welche dieser Variablen sind miteinander korreliert?

1. LSTAT: RM (-0.61), INDUS (0.60), NOX (0.59)
2. RM: LSTAT (-0.61)
3. PTRATIO: -
4. INDUS: LSTAT (0.60), TAX (0.72), NOX (0.76)
5. TAX: INDUS (0.72), NOX (0.67)
6. NOX: LSTAT (0.59), INDUS (0.76), TAX (0.67)

Welche sind daher Kandidaten,
die man evtl. weglassen könnte?

1. INDUS
2. NOX
3. (RM)

### c)  Dokumentation der eingesetzten Systemkonfiguration und Paketversionen
Dient dazu ein Mindestmaß an Reproduzierbarkeit sicherzustellen. (Packet in anaconda prompt installiert mit: conda install -c conda-forge version_information)

In [ ]:
%reload_ext version_information
%version_information
%version_information numpy , pandas

## Aufgabe 2

### a)
Laden Sie die Datenbank als zip-File aus Moodle herunter und lesen Sie dieses File direkt
in ein neues Notebook ein. Die bisher verwendete Pandas-Methode read_csv() funktioniert
für JSON-Files leider nicht. Das heruntergeladene File wird stattdessen mithilfe des Py-
thonmoduls zipfile entpackt und dem Python-Befehl open() eingelesen. Die Umwandlung
des JSON-Formates in ein geeignetes Python-Format erfolgt mit einem weiteren Modul der
Python-Standardlibrary, json, hier mithilfe der Funktion json.load(). Lesen Sie dazu die
zugehörigen, auf dem Web bzw. Stackoverflow verfügbaren Anleitungen.

In [ ]:
import json
import zipfile
import os
zf = zipfile.ZipFile("./foods-2011-10-03.json.zip")
with zf.open(zf.namelist()[0]) as f:
    data = json.load(f)
    
foods = pd.DataFrame(data)

### b)
Die Datenbank steht nun in Form einer Liste aus 6636 Python-Dictionaries zu Verfügung.
Jedes Dictionary enthält Angaben zu einem Nahrungsmittel. Greifen Sie sich ein beliebiges
Nahrungsmittel heraus und lassen sich die Namen der Einträge mit der Methode dict.keys()
anzeigen. Einer der Einträge enthält die enthaltenen Nährstoffe (’nutrients’), ebenfalls als
Dictionary. Lassen Sie sich wiederum einen beliebigen Eintrag der Nährstoffliste anzeigen.
Es sollte auffallen, dass manche Feldnamen doppelt vorkommen.

In [ ]:
foods.head()

In [ ]:
foods.loc[[0]]

In [ ]:
foods.loc[[0]].keys()

In [ ]:
foods.loc[0]["nutrients"][0]

-> Der Feldname "groups" oder "description" kommt beispielsweise doppelt vor!

Teile dieser hierarchischen Struktur sollen nun in eine einheitliche Tabelle umgewandelt
werden, um eine explorative Analyse durchführen zu können.
Vorgehensweise:
- Kopieren Sie zunächst die Felder ’description’,’group’,’id’,’manufacturer’ in einen eigenen DataFrame info, sowie alle Nährstofflisten in ein Array von DataFrames, wobei Sie an jeden DataFrame die entsprechende ID des Nahrungsmittels als eigene Spalte anhängen.
- Dieses Array wird mithilfe der Funktion pandas.concat() zu einem großen DataFrame nutrients (389355 Einträge) vereinigt.
- Entfernen Sie alle Duplikate aus diesem DataFrame.
- Bevor beide DataFrames vereinigt werden können, gibt es noch ein Problem: beide enthalten Felder mit dem Namen ’description’ und ’group’ (s.o.). Benennen Sie diese daher mithilfe von DataFrame.rename() in eindeutige Namen um.
- Vereinigen Sie beide DataFrames mit pandas.merge(nutrients, info, on=’id’, how=’outer’) anhand der Nahrungsmittel-ID. 

Überprüfen Sie das Ergebnis jeder Manipulation mit DataFrame.head().

In [ ]:
info_data = {'description':[],'group':[],'id':[],'manufacturer':[]}
nutrients_data = {'id': [], 'value': [], 'units': [], 'description': [], 'group': []}

food = 0
while food <= 6635:
    info_data["description"].append(foods.loc[food].description)
    info_data["group"].append(foods.loc[food].group)
    info_data["id"].append(foods.loc[food].id)
    info_data["manufacturer"].append(foods.loc[food].manufacturer)
    
    food_id = foods.loc[food].id
    for nutrient in foods.loc[food].nutrients:
        nutrients_data["id"].append(food_id)
        nutrients_data["value"].append(nutrient["value"])
        nutrients_data["units"].append(nutrient["units"])
        nutrients_data["description"].append(nutrient["description"])
        nutrients_data["group"].append(nutrient["group"])
    food = food + 1

In [ ]:
df_info = pd.DataFrame(info_data)
display(df_info.head())

df_info = df_info.rename(columns={'description':'food_description', 'group':'food_group'})
display(df_info.head())

df_nutrients = pd.DataFrame(nutrients_data)
display(df_nutrients.head())

df_nutrients = df_nutrients.rename(columns={'description':'nutri_description', 'group':'nutri_group'})
display(df_nutrients.head())

df_nutrients = df_nutrients.drop_duplicates()
display(df_nutrients.head())

merge = pd.merge(df_nutrients, df_info, on='id', how='outer')
display(merge)

### c)
Nun sind die Daten bereit für die Untersuchung auf das Spurenelement Zink (Feldname:
’Zinc, Zn’). Lesen Sie dazu alle Tabelleneinträge mithilfe einer geeigneten Indizierung
in einen DataFrame aus, der nur Einträge zum Nährstoff Zink enthält.

In [ ]:
df_zink = merge[(merge.nutri_description == "Zinc, Zn")]
display(df_zink.head())

Daraus wählen
Sie wiederum die Spalte mit dem Zinkgehalt in mg (’value’) aus und stellen dafür ein
Histogramm und eine Liste deskriptiver Statistiken dar.

In [ ]:
import matplotlib.pyplot as plt
df_grouped_mean_zink = df_zink.groupby(["food_group"]).mean()
display(df_grouped_mean_zink.hist(column="value", bins=df_grouped_mean_zink.size))

df_zink_sorted = df_zink.sort_values("value")
display(df_zink_sorted.head())

display(df_zink_sorted.hist(column="value", bins=100))
df_zink.describe()

Finden Sie in Ihrer Tabelle Edamer
(’Cheese, edam’). Hat Edamer einen überdurchschnittlichen Zinkgehalt?

In [ ]:
df_cheese = df_zink[(df_zink.food_description == "Cheese, edam")]
zink_mean = df_zink["value"].mean()
zink_cheese = df_cheese.iloc[0]["value"]
print("Zink gehalt von Cheese, edam:", zink_cheese)
print("Durschnittlicher Zink gehalt:", zink_mean)
print("Hat Edammer einen überdurchschnittlichen Zinkgehalt? ", zink_cheese >= zink_mean)

Haben mehr als 75% aller Nahrungsmittel einen kleineren Zinkgehalt?

In [ ]:
print(df_zink.quantile(0.75).value < zink_cheese)

Welches Nahrungsmittel hat den
maximalen Zinkgehalt?

In [ ]:
print(df_zink.loc[df_zink["value"].idxmax()].food_description)